In [ ]:
!pip install --upgrade --force-reinstall --no-deps kaggle
!mkdir ~/.kaggle
!touch ~/.kaggle/kaggle.json

api_token = {"username":"yishengliu","key":"600d4a6327f17945c27ff54c8bae7bad"}

import json

with open('/root/.kaggle/kaggle.json', 'w') as file:
    json.dump(api_token, file)

!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle competitions download -c challenges-in-representation-learning-facial-expression-recognition-challenge

In [ ]:
!unzip /content/challenges-in-representation-learning-facial-expression-recognition-challenge.zip
!rm -rf /content/train.csv /content/test.csv /content/icml_face_data.csv /content/example_submission.csv /content/challenges-in-representation-learning-facial-expression-recognition-challenge.zip
!tar -xzf /content/fer2013.tar.gz
!cd /content
!mkdir dataset/fer2013
!mv /content/fer2013/fer2013.csv /content/dataset/fer2013
!rm -rf /content/fer2013 /content/fer2013.tar.gz

In [ ]:
import pandas as pd
import torch
import numpy as np
import cv2

In [55]:
class dataset_provider(object):
    def __init__(self, dataset_name, image_size):
        self.dataset_name = dataset_name
        self.image_size = image_size
        self.dataset_path = '/content/dataset/fer2013/fer2013.csv'

    def dataset_initializer(self):
        data = pd.read_csv(self.dataset_path)
        pixels = data['pixels'].tolist()
        width = 48
        height = 48
        faces = []
        for pixel_sequence in pixels:
            face = [int(pixel) for pixel in pixel_sequence.split(' ')]
            face = np.asarray(face).reshape(width, height)
            face = cv2.resize(face.astype('uint8'), self.image_size)
            faces.append(face.astype('float32'))
        faces = np.asarray(faces)
        faces = np.expand_dims(faces, -1)
        emotions = pd.get_dummies(data['emotion']).values
        return faces, emotions

def get_labels(dataset_name):
    if dataset_name == 'fer2013':
        return {0: 'angry', 1: 'disgust', 2: 'fear', 3: 'happy', 4: 'sad', 5: 'surprise', 6: 'neutral'}
    else:
        raise Exception('Invalid args: Wrong dataset name.')


def get_class_to_arg(dataset_name):
    if dataset_name == 'fer2013':
        return {'angry': 0, 'disgust': 1, 'fear': 2, 'happy': 3, 'sad': 4, 'surprise': 5, 'neutral': 6}
    else:
        raise Exception('Invalid args: Wrong dataset name.')


def pre_process(x, _is=True):
    x = x.astype('float32')
    x = x / 255.0
    if _is:
        x = x - 0.5
        x = x * 2.0
    return x



In [57]:
input_shape = (64, 64, 1)
dataset = dataset_provider('fer2013', image_size=input_shape[:2])
faces, emotions = dataset.dataset_initializer()
faces = pre_process(faces)
n_train = int(0.8 * len(faces))
n_test = len(faces) - n_train
rng = torch.Generator().manual_seed(291)
ds_train_faces, ds_test_faces = torch.utils.data.random_split(faces, [n_train, n_test], rng)
ds_train_emotions, ds_test_emotions = torch.utils.data.random_split(emotions, [n_train, n_test], rng)